# Cortex - M3 Example

In [1]:
from pynq import Overlay
import time 

class M3Wrapper:
    # constant
    __M3_BITSTREAM_PATH: str = './m3.bit'
    __M3_EXECUTABLE_BIN_PATH: str = './m3.bin'
    
    __M3_EXTERNAL_DATA_START_ADDR = 0x20100000
    
    def __init__(self):
        self.overlay = Overlay(self.__M3_BITSTREAM_PATH)

    def reset(self):
        self.overlay.axi_pl_soft_reset.write(0x0, 0)

    def release(self):
        self.overlay.axi_pl_soft_reset.write(0x0, 1)

    def load_bin_to_m3(self, bin_file: str = str(), verbose: bool = False):
        if bin_file:
            bin_file_path = binfile
        else:
            bin_file_path = self.__M3_EXECUTABLE_BIN_PATH
        
        bin_code = open(bin_file_path, 'rb')

        addr = 0x0
        while True:
            word = bin_code.read(4)

            if not word:
                break
            else:
                self.overlay.axi_bram_ps_code.write(addr, word)
                addr += 4

        bin_code.close()
        
        if verbose:
            print(f'Words wrote are {addr-4}')
    
    def run_for_seconds(self, reset_time: float):
        self.release() # release M3
        time.sleep(reset_time)
        self.reset() # reset M3
    
    
    def dump_external_sram_non_zero(self):
        
        high_idx = self.overlay.axi_bram_ps_data.size

        idx = 0
        while (idx < high_idx):
            data = m3.overlay.axi_bram_ps_data.read(idx)

            if data:
                print(f'addr:0x{idx+self.__M3_EXTERNAL_DATA_START_ADDR:08x}, value: 0x{data:08x}')

            idx += 4

In [2]:
m3 = M3Wrapper()

In [3]:
m3.load_bin_to_m3(verbose=True)

Words wrote are 292


In [4]:
m3.run_for_seconds(20.0)

In [5]:
m3.dump_external_sram_non_zero() # see ResetISR in src/reset_handler.s

addr:0x20100004, value: 0xdeadc0fe
addr:0x20100008, value: 0x00000128
addr:0x2010000c, value: 0x00000128
addr:0x20100010, value: 0x20000008
addr:0x20100014, value: 0x00000002
addr:0x20100040, value: 0x00000120
addr:0x20100044, value: 0x00000120
addr:0x20100048, value: 0x10000020
addr:0x2010004c, value: 0x00000008
addr:0x2013fff4, value: 0x20100050
addr:0x2013fff8, value: 0x000000a1
